## Extracting Context Topics and Contexts

In [ ]:
import json
import pandas as pd
import numpy as np
from collections import defaultdict


np.random.seed(42)
# load from parquet file
df = pd.read_parquet('../../data/pquad/test_data.parquet')

In [17]:
df.paragraphs[0][2]["context"]

'پاتریشیا کرون، مایکل کوک و بسیاری دیگر از محققان، بر اساس متن و تحقیقات باستان\u200cشناسی، تصور کرده\u200cاند که «مسجدالحرام» در مکه واقع نشده\u200cاست، بلکه در شمال غربی شبه جزیره عربستان واقع شده\u200cاست. دن گیبسون اظهار داشت که اولین مساجد اسلامی و جهت\u200cهای گورستان (قبله) نشان داد پترا، محمد اولین افشاگری\u200cهای خود را در اینجا دریافت کرد و اسلام در اینجا برقرار شد. رابرت برترام سرژانت در ژورنال جامعهٔ شرق\u200cشناسی آمریکا این نظریه را یک جدال سردرگم و غیرمنطقی توصیف کرد که به دلیل درک نادرست پاتریشیا کرون از متون عربی، عدم آشنایی با جامعهٔ عربستان، و سوءتعبیر از نوشته\u200cهای دیگر به نفع نظریهٔ خود پیچیده\u200cتر شده\u200cاست.'

In [18]:
len(df.paragraphs)

114

In [20]:
len(df.paragraphs[3])

5

In [23]:
# Extract topics from context to new column, meanwhile, extracting the context themselves into a list
all_context = []

def extract_topics(df):
    global all_context
    for i in range(len(df.paragraphs)):
        for j in range(len(df.paragraphs[i])):
            all_context.append(df.paragraphs[i][j]["context"])
    
extract_topics(df)
    
all_context = list(set(all_context))  # Remove duplicates from contexts
print(f"Total unique contexts: {len(all_context)}")


Total unique contexts: 1059


In [25]:
## save all_context to a json file
with open('all_context.json', 'w', encoding='utf-8') as f:
    json.dump(all_context, f, ensure_ascii=False, indent=4)

# Extracting Knowledge Graph Triplets

In [26]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List
import dotenv
import json
import os

In [27]:
dotenv.load_dotenv()

MODEL_NAME = "gpt-4o-mini"

llm = ChatOpenAI(
    model=MODEL_NAME,
    temperature=0,
    api_key=os.getenv("METIS_API_KEY"),
    base_url="https://api.metisai.ir/openai/v1",
)

In [29]:
# load all_context from json file
with open('all_context.json', 'r', encoding='utf-8') as f:
    all_context = json.load(f)

# print 3 first contexts
print("First 3 contexts:")
for context in all_context[:3]:
    print(context)
    print("-" * 10)

First 3 contexts:
ایرانیان از ۱۳۱۶ این دریا را مازندران می‌نامند. نام دریای مازندران و دریای خزر در ۵۰ سال گذشته در رسانه‌های گروهی ایران رایج بوده‌است. در سال ۱۳۶۱ دولت نام دریای مازندران را نام رسمی اعلام کرد.شاهزاده مسعود میرزا ظل‌السلطان ملقب به ظل السلطان که از شاهزادگان قاجار و بزرگترین پسر به سن بلوغ رسیده ناصر الدین شاه بود که در دوران جوانی مدتی حاکم مازندران بود. وی در خاطرات خود آورده‌است: وقتی در بلده در اردوی همایونی بودم، دستورالعملی به جناب بهاء الملک وزیر من و من مرحمت فرمودند که میانکاله را ضبط کرده قلعه ای بسازیم، و در او ساخلو بگذاریم. این میانکاله همان شبه جزیره آبسکون است که شرحش را نوشتیم. یکی از اسامی این دریای مازندران را دریای آبسکون نیز می‌گویند.
----------
پیروزی رونالد ریگان در ابتدای دهه ۱۹۸۰ در حقیقت پیروزی جناح راست حزب جمهوری‌خواه بود که محافظه‌کاری را به شکل عمیق در کشور حاکم کرد. وی با شعار کاهش مالیات‌ها، کاهش هزینه‌های دولت، تقویت بودجه نظامی و کاهش دخالت‌های دولت در امر بخش خصوصی به کاخ سفید وارد شد. به همین دلیل در نخستین دوره ریاست جمهوری ریگان

In [30]:
class NamedEntities(BaseModel):
    """A structured list of named entities."""
    named_entities: List[str] = Field(
        ...,
        description="A list of named entities extracted from the provided passage."
    )

class KnowledgeTriple(BaseModel):
    """Represents a single Subject-Predicate-Object relationship."""
    subject: str = Field(..., description="The subject of the relationship.")
    predicate: str = Field(..., description="The predicate or verb phrase describing the relationship.")
    object: str = Field(..., description="The object of the relationship.")

class Triples(BaseModel):
    """A structured list of knowledge triples."""
    triples: List[KnowledgeTriple] = Field(
        ...,
        description="A list of knowledge triples (subject, predicate, object) extracted from the passage."
    )

In [31]:
ner_system_prompt = """
Instruction:
Your task is to extract named entities from the given paragraph in the user's message.
Respond with a JSON list of entities.

One-Shot Demonstration:
If the user provides the paragraph:
"Radio City is India's first private FM radio station and was started on 3 July 2001. It plays Hindi, English and regional songs. Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features."

Your output should be a JSON object with a list of the extracted entities:
{{"named_entities": ["Radio City", "India", "3 July 2001", "Hindi", "English", "May 2008", "PlanetRadiocity.com"]}}
"""

# System Prompt for Open Information Extraction (OpenIE)
# This example is taken from Figure 9 of the HippoRAG paper.
openie_system_prompt = """
Instruction:
Your task is to construct an RDF (Resource Description Framework) graph from the given passages and named entity lists.
Respond with a JSON list of triples, with each triple representing a relationship in the RDF graph.
Pay attention to the following requirements:
- Each triple should contain at least one, but preferably two, of the named entities in the list for each passage.
- Clearly resolve pronouns to their specific names to maintain clarity.

One-Shot Demonstration:
If the user provides the paragraph and entity list:
Paragraph:
"Radio City is India's first private FM radio station and was started on 3 July 2001. It plays Hindi, English and regional songs. Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features."
Named Entity List:
["Radio City", "India", "3 July 2001", "Hindi", "English", "May 2008", "PlanetRadiocity.com"]

Your output should be a JSON object containing a list of the extracted triples:
{{"triples": [
    ["Radio City", "located in", "India"],
    ["Radio City", "is", "private FM radio station"],
    ["Radio City", "started on", "3 July 2001"],
    ["Radio City", "plays songs in", "Hindi"],
    ["Radio City", "plays songs in", "English"],
    ["Radio City", "forayed into", "New Media"],
    ["Radio City", "launched", "PlanetRadiocity.com"],
    ["PlanetRadiocity.com", "launched in", "May 2008"],
    ["PlanetRadiocity.com", "is", "music portal"],
    ["PlanetRadiocity.com", "offers", "news"],
    ["PlanetRadiocity.com", "offers", "videos"],
    ["PlanetRadiocity.com", "offers", "songs"]
]}}
"""

In [32]:
ner_prompt = ChatPromptTemplate.from_messages([
    ("system", ner_system_prompt),
    ("human", "{passage}")
])

openie_prompt = ChatPromptTemplate.from_messages([
    ("system", openie_system_prompt),
    ("human", "Paragraph:\n{passage}\n\nNamed Entity List:\n{entities}")
])

In [33]:
structured_ner_llm = llm.with_structured_output(NamedEntities)
structured_openie_llm = llm.with_structured_output(Triples)

ner_chain = ner_prompt | structured_ner_llm
openie_chain = openie_prompt | structured_openie_llm

In [34]:
def extract_knowledge_structured(passage: str) -> (List[str], List[dict]):
    """
    Implements the two-step HippoRAG extraction process using a system/user prompt structure.
    """
    try:
        ner_response = ner_chain.invoke({"passage": passage})
        entities = ner_response.named_entities
        print(f"  -> Extracted Entities: {entities}")
    except Exception as e:
        print(f"  -> Error during structured NER extraction: {e}")
        return [], []

    if not entities:
        print("  -> No entities found, skipping triple extraction.")
        return [], []

    try:
        entities_str = json.dumps(entities, ensure_ascii=False)
        openie_response = openie_chain.invoke({"passage": passage, "entities": entities_str})
        triples = [triple.model_dump() for triple in openie_response.triples]
        print(f"  -> Extracted Triples: {triples}")
        return entities, triples
    except Exception as e:
        print(f"  -> Error during structured OpenIE extraction: {e}")
        return entities, []

In [35]:
## Test 5 contexts
contexts_to_process = all_context[:5]
knowledge_graph_data = []

print("Starting Knowledge Graph Extraction (HippoRAG Paper Examples)")
for i, context_passage in enumerate(contexts_to_process):
    print(f"\ncontext {i+1}/5:")
    print(f"'{context_passage}'")

    extracted_entities, extracted_triples = extract_knowledge_structured(context_passage)

    if extracted_triples:
        knowledge_graph_data.append({
            "id": i,
            "passage": context_passage,
            "entities": extracted_entities,
            "triples": extracted_triples
        })
    print("-" * 20)

Starting Knowledge Graph Extraction (HippoRAG Paper Examples)

context 1/5:
'ایرانیان از ۱۳۱۶ این دریا را مازندران می‌نامند. نام دریای مازندران و دریای خزر در ۵۰ سال گذشته در رسانه‌های گروهی ایران رایج بوده‌است. در سال ۱۳۶۱ دولت نام دریای مازندران را نام رسمی اعلام کرد.شاهزاده مسعود میرزا ظل‌السلطان ملقب به ظل السلطان که از شاهزادگان قاجار و بزرگترین پسر به سن بلوغ رسیده ناصر الدین شاه بود که در دوران جوانی مدتی حاکم مازندران بود. وی در خاطرات خود آورده‌است: وقتی در بلده در اردوی همایونی بودم، دستورالعملی به جناب بهاء الملک وزیر من و من مرحمت فرمودند که میانکاله را ضبط کرده قلعه ای بسازیم، و در او ساخلو بگذاریم. این میانکاله همان شبه جزیره آبسکون است که شرحش را نوشتیم. یکی از اسامی این دریای مازندران را دریای آبسکون نیز می‌گویند.'
  -> Extracted Entities: ['ایرانیان', '۱۳۱۶', 'دریای مازندران', 'دریای خزر', '۵۰ سال گذشته', 'رسانه\u200cهای گروهی ایران', '۱۳۶۱', 'دولت', 'شاهزاده مسعود میرزا ظل\u200cالسلطان', 'ظل السلطان', 'شاهزادگان قاجار', 'ناصر الدین شاه', 'بلده', 'اردوی همایونی', 'ج

In [39]:
# Now do the same for all of the contexts
from tqdm import tqdm

def extract_knowledge_structured(passage: str) -> (List[str], List[dict]):
    """
    Implements the two-step HippoRAG extraction process using a system/user prompt structure.
    """
    try:
        ner_response = ner_chain.invoke({"passage": passage})
        entities = ner_response.named_entities
        # print(f"  -> Extracted Entities: {entities}")
    except Exception as e:
        print(f"  -> Error during structured NER extraction: {e}")
        return [], []

    if not entities:
        print("  -> No entities found, skipping triple extraction.")
        return [], []

    try:
        entities_str = json.dumps(entities, ensure_ascii=False)
        openie_response = openie_chain.invoke({"passage": passage, "entities": entities_str})
        triples = [triple.model_dump() for triple in openie_response.triples]
        # print(f"  -> Extracted Triples: {triples}")
        return entities, triples
    except Exception as e:
        print(f"  -> Error during structured OpenIE extraction: {e}")
        return entities, []
    
    

knowledge_graph_data = []

print("Starting Knowledge Graph Extraction (only 300 context from pquad dataset)")
## we took 300 contexts from pquad dataset, while our test questions only require 32 contexts
# this is only because of the budget limit of the API calls

i = 0
for context_passage in tqdm(all_context[:300]):
    extracted_entities, extracted_triples = extract_knowledge_structured(context_passage)

    if extracted_triples:
        knowledge_graph_data.append({
            "id": i,
            "passage": context_passage,
            "entities": extracted_entities,
            "triples": extracted_triples
        })
        
    i += 1


Starting Knowledge Graph Extraction (only 300 context from pquad dataset)


100%|██████████| 300/300 [44:53<00:00,  8.98s/it]


In [40]:
# save the knowledge graph data to a json file
with open('knowledge_graph_triples_structured.json', 'w', encoding='utf-8') as f:
    json.dump(knowledge_graph_data, f, ensure_ascii=False, indent=4)

# Check prepared data

In [41]:
import pandas as pd
import igraph as ig
import json
from collections import defaultdict

In [42]:
with open('knowledge_graph_triples_structured.json', 'r', encoding='utf-8') as f:
    knowledge_graph_data = json.load(f)
df_kg = pd.DataFrame(knowledge_graph_data).drop(columns=['id'])
df_kg.head()

,passage,entities,triples
0,ایرانیان از ۱۳۱۶ این دریا را مازندران می‌نامند...,"[ایرانیان, ۱۳۱۶, دریای مازندران, دریای خزر, ۵۰...","[{'subject': 'ایرانیان', 'predicate': 'نام این..."
1,پیروزی رونالد ریگان در ابتدای دهه ۱۹۸۰ در حقیق...,"[رونالد ریگان, دهه ۱۹۸۰, جناح راست حزب جمهوری‌...","[{'subject': 'رونالد ریگان', 'predicate': 'پیر..."
2,محمد غفاری در خانواده‌ای هنرمند و سرشناس در رو...,"[محمد غفاری, کله, کاشان, ابوتراب غفاری, تهران,...","[{'subject': 'محمد غفاری', 'predicate': 'born ..."
3,منطقه گیلان دارای بارش فراوان است تا جایی که ب...,"[گیلان, ۱۲۰ سانتی‌متر, چراغ علی تپه, دهانه گوه...","[{'subject': 'گیلان', 'predicate': 'has annual..."
4,فارسی از گذشته‌های دور، به ویژه از دوره یوآن ...,"[فارسی, دوره یوان, سده بیستم, چین, ایران, ساسا...","[{'subject': 'فارسی', 'predicate': 'had import..."


In [44]:
all_named_entities = set()
all_triples = []
for entity_list in df_kg['entities']:
    all_named_entities.update(entity_list)
    
for triples in df_kg['triples']:
    for triple in triples:
        all_triples.append((triple['subject'], triple['predicate'], triple['object']))
    
print(f"Total unique named entities: {len(all_named_entities)}")
print(f"Total unique triples: {len(all_triples)}")

Total unique named entities: 3397
Total unique triples: 4329


In [45]:
all_named_entities

{'بدیهی گرایی فلسفی',
 'هسته',
 'آتئیست',
 'فیلم',
 'مک\u200cفارلین',
 'ALGOL ۶۸',
 '۱۹۴۲',
 'تغییر آبی',
 'آسیایی ۱۹۹۴',
 'آبشار نیاگارا',
 '+Google',
 '۶۳۹ میلیارد دلار',
 'بعد از جنگ',
 'دهه ۹۰',
 'شاخه غربی زبان\u200cهای ایرانی',
 'تیٔاتر',
 'زلاتان ابراهیموویچ',
 'آزمایشگاه\u200cهای بل',
 '۵۰۰ هزار نفر',
 'تب',
 'نفت سفید',
 'وبگاه\u200cهای شبکه\u200cهای اجتماعی',
 'لیگ برتر فصل ۸۳–۸۲',
 'سید روح\u200cالله خمینی',
 'فایفر',
 'پیج',
 'شلوار',
 'بی\u200cخدایی قوی',
 '۲۷ کشور اروپایی',
 'ترکان',
 'مسیحیان',
 'مسیحیت',
 'بلاک پارتی',
 'آزمایش سکوی پرتاب',
 'گوسفند وحشی',
 'قاجار',
 'ندانم\u200cگرایان',
 'سناتور هیوبرت هامفری',
 'کوه قارن',
 'رابرت برترام سرژانت',
 '۲۹ سال',
 'موازی',
 '۱۲۶٬۴۷۶٬۰۰۰ نفر',
 'تولید مواد شیمیایی',
 'شیخ کلینی',
 'Version Control',
 '۳۲ تیم',
 'قباد اول',
 'فانک',
 '۲۰۰۰۰ سال پیش',
 'هیوستون',
 'آغاجاری',
 'مراد بخش',
 'پَرِ مرغ',
 'اردیبهشت ۱۳۵۹',
 'اووزان',
 'اینترنت پروتکل اینترنت',
 'semi-finals',
 'ایلین وورنوس',
 'اشکال\u200cزدایی',
 'اردوان پنجم',
 